# Google Scholar Profile

In [29]:
import pandas as pd
import time
pd.set_option('display.max_colwidth',200)

* Export the data into a CSV file. 
* Copy and paste the web version of the publication list into a spreadsheeh

In [191]:
import re
import unidecode

In [47]:
sp='https://docs.google.com/spreadsheets/d/e'
gs_id='2PACX-1vQ8oE7rkkqQ07WzSEq8qCciqwJu8lOx8cnH0YTn2e6HDL3DJHYF2KahjkuVv6wVSMJEPeJLfBLxRRub'
cp_id='2PACX-1vRKmxmnaxySZ7OMUNbH-BpaGB1RILM55CMfJtlJUd6XUwnrEAzUh5YkzQI7R8_fl-kRHSBpGKfXwC4O'
gs=pd.read_csv('{}/{}/pub?output=csv'.format(sp,gs_id)).reset_index(drop=True)
time.sleep(1)
cp=pd.read_csv('{}/{}/pub?output=csv'.format(sp,cp_id)).reset_index(drop=True)

Normalize data

In [216]:
gs['TitleU']=gs['Title'].apply(lambda s: unidecode.unidecode(s) if s else s)
cp['Reference']=cp['Reference'].apply(lambda s: unidecode.unidecode(s) if s else s)

In [406]:
author='[A-Z]{1,3}\s[A-Z][a-z\-]+'
AUTHOR='[A-Z]{1,3}\s[A-Z\-]+'
cp['TitleU']=cp['Reference'].apply(lambda s: re.sub(
    '([a-z0-9…\?\.\'\"\]\)])%s,{0,1}' %author,r'\1::' ,s,re.UNICODE)).str.split('::').str[0]
#Special cases
#...
#cp['TitleU']=cp['TitleU'].apply(lambda s: re.sub(
#                  '(\.\.\.)%s,{0,1}' %author,r'\1::' ,s,re.UNICODE)
#                   if re.search('\.\.\.%s[,\n]' %author,s,re.UNICODE) else s
#                  ).str.split('::').str[0]
#Full upper-case letter: NOT POSSIBLE TO DISCRIMINATE FINAL 1 or two \w
cp['TitleU']=cp['TitleU'].apply(lambda s: re.sub('(\w+)[A-Z]\s[A-Z][a-z\-]+',r'\1::',s,re.UNICODE) 
                  if re.search('\s%s[,\n]' %author,s,re.UNICODE) else s
                 ).str.split('::').str[0]
# Upper case last name, only if end in , or \n
cp['TitleU']=cp['TitleU'].apply(lambda s: re.sub('(\w+)[A-Z]\s[A-Z][A-Z\-]+',r'\1::',s,re.UNICODE) 
                  if re.search('[\s\w]%s[,\n]' %AUTHOR,s,re.UNICODE) else s
                 ).str.split('::').str[0]
# Separated by space
cp['TitleU']=cp['TitleU'].apply(lambda s: re.sub(
                '(\w)\s%s,{0,1}' %author,r'\1::' ,s,re.UNICODE      )                
                   if re.search('\s%s[,\n]' %author,s,re.UNICODE) else s
                  ).str.split('::').str[0]
#Long authors: L de la Torre
longauthor='[A-Z]{1,3}(\s[a-z]+)+\s[A-Z][a-z\-]+'
cp['TitleU']=cp['TitleU'].apply(lambda s: re.sub(
            '([a-z0-9…\?\.\'\"\]\)])%s,{0,1}' %longauthor,r'\1::' ,s,re.UNICODE)
                   if re.search('%s[,\n]' %longauthor,s,re.UNICODE) else s
                  ).str.split('::').str[0]



#cp['TitleU']=cp['Reference'].apply(lambda s: re.sub(
#    '(\.\.\.)[A-Z]+\s[A-Z][a-z\-]+,{0,1}',r'\1::',s,re.UNICODE)).str.split('::').str[0]
#Title in Caps
#Totle end with space
#Fix obvious problems
cp['TitleU']=cp['TitleU'].str.replace('([0-9\s][TGM]e)$',r'\1V')

In [407]:
GS=gs.merge(cp,on='TitleU',how='left')

In [408]:
GS[~GS.Reference.isna()].shape

(1747, 12)

TODO: Make a similarity search for each of the GF.TitleU  `NaN`
with the list of cp.TitleU

In [409]:
GS['Cites']=GS['Cites'].apply(lambda s: int(s.replace('*','')) if isinstance(s,str) else 0)

In [413]:
GS

,Authors,Title,Publication,Volume,Number,Pages,Year_x,Publisher,TitleU,Reference,Cites,Year_y
0,"Samia, A; Feddi, E; Duque, CA; Mora-Ramos, ME; Akimov, V; Correa, JD;",Optoelectronic properties of phosphorene quantum dots functionalized with free base porphyrins,Computational Materials Science,171.0,NaN,109278,2020.0,Elsevier,Optoelectronic properties of phosphorene quantum dots functionalized with free base porphyrins,"Optoelectronic properties of phosphorene quantum dots functionalized with free base porphyrinsA Samia, E Feddi, CA Duque, ME Mora-Ramos, V Akimov, JD Correa\nComputational Materials Science 171, 1...",0,2020.0
1,"Calle, Julián; Restrepo, Diego; Zapata, Óscar;",Dirac neutrino mass generation from a Majorana messenger,Physical Review D,101.0,3,35004,2020.0,American Physical Society,Dirac neutrino mass generation from a Majorana messenger,"Dirac neutrino mass generation from a Majorana messengerJ Calle, D Restrepo, O Zapata\nPhysical Review D 101 (3), 035004",1,2020.0
2,"Petelczyc, Krzysztof; Kolodziejczyk, Andrzej; Błocki, Narcyz; Byszewska, Anna; Jaroszewicz, Zbigniew; Kakarenko, Karol; Kołacz, Katarzyna; Miler, Michał; Mira-Agudelo, Alejandro; Torres-Sepúlveda,...",Model of the light sword intraocular lens: in-vitro comparative studies,Biomedical Optics Express,11.0,1,40-54,2020.0,Optical Society of America,Model of the light sword intraocular lens: in-vitro comparative studies,"Model of the light sword intraocular lens: in-vitro comparative studiesK Petelczyc, A Kolodziejczyk, N Blocki, A Byszewska, Z Jaroszewicz, ...\nBiomedical Optics Express 11 (1), 40-54",0,2020.0
3,"Mazo-Vasquez, JD; Hincapie-Zuluaga, LM; Reyes-Gómez, E;",Refractive index and group velocity of electromagnetic modes in a relativistic Fermi gas at finite temperatures,JOSA B,37.0,2,211-217,2020.0,Optical Society of America,Refractive index and group velocity of electromagnetic modes in a relativistic Fermi gas at finite temperatures,"Refractive index and group velocity of electromagnetic modes in a relativistic Fermi gas at finite temperaturesJD Mazo-Vasquez, LM Hincapie-Zuluaga, E Reyes-Gomez\nJOSA B 37 (2), 211-217",0,2020.0
4,"Tiutiunnyk, A; Pérez-Quintana, I; Laroze, D; Duque, CA; Mora-Ramos, ME;",Influence of conduction-band non-parabolicity on terahertz intersubband Raman gain in GaAs/InGaAs step asymmetric quantum wells,Applied Physics A,126.0,1,23,2020.0,Springer Berlin Heidelberg,Influence of conduction-band non-parabolicity on terahertz intersubband Raman gain in GaAs/InGaAs step asymmetric quantum wells,"Influence of conduction-band non-parabolicity on terahertz intersubband Raman gain in GaAs/InGaAs step asymmetric quantum wellsA Tiutiunnyk, I Perez-Quintana, D Laroze, CA Duque, ME Mora-Ramos\nAp...",0,2020.0
5,"Sucerquia, Mario; Ramírez, Vanesa; Alvarado-Montes, Jaime A; Zuluaga, Jorge I;",Can close-in giant exoplanets preserve detectable moons?,Monthly Notices of the Royal Astronomical Society,492.0,3,3499-3508,2020.0,Oxford University Press,Can close-in giant exoplanets preserve detectable moons?,"Can close-in giant exoplanets preserve detectable moons?M Sucerquia, V Ramirez, JA Alvarado-Montes, JI Zuluaga\nMonthly Notices of the Royal Astronomical Society 492 (3), 3499-3508",0,2020.0
6,"Vinasco, JA; Radu, A; Tiutiunnyk, A; Restrepo, RL; Laroze, D; Feddi, E; Mora-Ramos, ME; Morales, AL; Duque, CA;",Revisiting the adiabatic approximation for bound states calculation in axisymmetric and asymmetrical quantum structures,Superlattices and Microstructures,138.0,NaN,106384,2020.0,Academic Press,Revisiting the adiabatic approximation for bound states calculation in axisymmetric and asymmetrical quantum structures,"Revisiting the adiabatic approximation for bound states calculation in axisymmetric and asymmetrical quantum structuresJA Vinasco, A Radu, A Tiutiunnyk, RL Restrepo, D Laroze, E Feddi, ...\nSuperl...",0,2020.0
7,"Hien, Nguyen D; Nguyen, Chuong V; Hieu, Nguyen N; Kubakaddi, SS; Duque, CA; Mora-Ramos, ME; Dinh, Le; Bich, Tran 

In [418]:
GS.to_json('GS.json')

TODO: Example

In [415]:
cp[cp.TitleU.str.contains('Measurement of the associated production o')]['TitleU']

7    Measurement of the associated production of a Z boson with charm or bottom quark jets in proton-proton collisions at  13 TeV
Name: TitleU, dtype: object

In [416]:
gs[gs.Title.str.contains('Measurement of the associated production o')].TitleU

13    Measurement of the associated production of a Z boson with charm or bottom quark jets in proton-proton collisions at $\sqrt {s}= $13 TeV
Name: TitleU, dtype: object

In [397]:
GS[['TitleU','Reference']].loc[19]

TitleU       Observation of the Lb0- J/psLph decay in proton-proton collisions at sqrt (s)= 13TeV
Reference                                                                                     NaN
Name: 19, dtype: object

In [405]:
cp[cp.Reference.str.contains('Observation of the')][['Reference','TitleU']].TitleU.str.replace('([0-9\s][TGM]e)$',r'\1V').loc[15]

'Observation of the Lb0- J/psLph decay in proton-proton collisions at sqrt (s)= 13TeV'